<a href="https://colab.research.google.com/github/hargurjeet/MachineLearning/blob/master/Random%20forest%20w/o%20pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# For training random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# !pip install opendatasets --quiet
# import opendatasets as od
# od.download('https://www.kaggle.com/c/30-days-of-ml/data')

In [6]:
# Load the training data
train = pd.read_csv("/content/30-days-of-ml/train.csv", index_col=0)
test = pd.read_csv("/content/30-days-of-ml/test.csv", index_col=0)

# Separate target from features
y = train['target']
X = train.drop(['target'], axis=1)

low_cardinality_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and X[cname].dtype == "object"]
numeric_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42)

my_cols = low_cardinality_cols + numeric_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()
X_test = test[my_cols].copy()


OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))


OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

OH_X_train = pd.concat([X_train[numeric_cols], OH_cols_train], axis=1)
OH_X_valid = pd.concat([X_valid[numeric_cols], OH_cols_valid], axis=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

%time
model = RandomForestRegressor(n_estimators=500, random_state=42)
model.fit(OH_X_train, y_train)
preds = model.predict(OH_X_valid)

print(mean_squared_error(y_valid, preds, squared=False))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
